<a href="https://colab.research.google.com/github/AHApiko/Simple-HR-chatbot-using-Langchain/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers langchain accelerate langchain_community

from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# Load a small legal model (Crumb/nano-mistral)
pipe = pipeline("text-generation", model="Crumb/nano-mistral")

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Ask a question
response = llm.invoke("What are the vacation rights for employees in Spain?")
print(response)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/340M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-1-b8c6afdbcb43>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


What are the vacation rights for employees in Spain?

The Spanish government has a long history of providing a safe and secure environment for employees to work


In [3]:
from langchain.document_loaders import TextLoader

# Load Estatuto de los Trabajadores
loader1 = TextLoader("estatuto_trabajadores_cleaned.txt")
estatuto_docs = loader1.load()

# Load Convenio de la Publicidad
loader2 = TextLoader("convenio_publicidad_cleaned.txt")
convenio_docs = loader2.load()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Define a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Small enough for focused search
    chunk_overlap=50,  # Ensures context is not lost
)

# Split and add metadata
estatuto_chunks = text_splitter.split_documents(estatuto_docs)
for doc in estatuto_chunks:
    doc.metadata["source"] = "Estatuto de los Trabajadores"

convenio_chunks = text_splitter.split_documents(convenio_docs)
for doc in convenio_chunks:
    doc.metadata["source"] = "Convenio de la Publicidad"

# Combine documents
all_documents = estatuto_chunks + convenio_chunks

In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Initialize free Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector database
vectorstore = FAISS.from_documents(all_documents, embeddings)

# Save FAISS database locally
vectorstore.save_local("legal_faiss_index")

# Perform similarity search
query = "What are the vacation rights for employees?"
docs = vectorstore.similarity_search(query, k=3)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
query = "What are the vacation rights for employees?"
docs = vectorstore.similarity_search(query, k=1)

# Print with source metadata
for doc in docs:
    print(f"Source: {doc.metadata['source']}\n")
    print(doc.page_content)
    print("-" * 50)


Source: Convenio de la Publicidad

El calendario de vacaciones se fijará en cada empresa con la Representación Legal de los Trabajadores. El trabajador conocerá las fechas que le correspondan dos meses antes, al menos, del comienzo de su disfrute.
El devengo de las vacaciones será proporcional al tiempo trabajado, computándose como mes completo la fracción del mismo.
--------------------------------------------------
